# **Invoke Bedrock model for SQL Query Generation**
## **1. Introduction**
- 이 노트북에서는 LLM을 사용하여 판매 데이터를 분석하기 위한 SQL 쿼리를 생성하는 방법을 보여드리겠습니다.
- 여기서는 Boto3 API를 사용하는 Bedrock의 Claude V2.1 모델을 사용하겠습니다.
- 이 예제에서 사용된 프롬프트는 프롬프트 이외의 텍스트 예제를 제공하지 않기 때문에 제로 샷 프롬프트라고 합니다.
> 참고: *`이 노트북은 AWS 환경 내부 또는 외부에서 실행할 수 있습니다.`*

### Context
- Amazon Bedrock의 SQL 코드 생성 기능을 시연하기 위해 Boto3 클라이언트를 사용하여 Amazon Bedrock API와 통신하는 방법을 살펴보겠습니다.
- 사용 가능한 다양한 구성과 간단한 입력이 어떻게 원하는 출력으로 이어질 수 있는지 보여드리겠습니다.

### Pattern
- 추가 예제를 제공하지 않고 (zero-shot) 작업, 명령어 및 내부 모델에 대한 입력으로 구성된 입력을 Amazon Bedrock API에 간단히 제공하여 출력을 생성합니다.
- 이 패턴의 목적은 강력한 LLM이 어떻게 당면한 작업을 쉽게 이해하고 매력적인 결과물을 생성하는지 보여주기 위한 것입니다.

### Use case
- 트렌드, 인기 제품 및 평균 매출에 초점을 맞춰 판매 데이터를 분석하기 위한 SQL 쿼리를 생성하는 사용 사례를 살펴보겠습니다.

### Persona
- Maya는 AnyCompany에서 주로 판매 및 재고 데이터에 집중하는 비즈니스 분석가입니다.
- 그녀는 스프레드시트 분석에서 데이터 기반 분석으로 전환하고 있으며, SQL을 사용하여 특정 데이터 요소를 효과적으로 가져오고자 합니다.
- 그녀는 분석을 위한 SQL 쿼리를 생성하는 데 LLM을 사용하고자 합니다.

### Imprementation
- 이 사용 사례를 충족하기 위해 이 노트북에서는 SQL 쿼리를 생성하는 방법을 보여드리겠습니다.
- 여기서는 Boto3 클라이언트와 함께 Amazon Bedrock API를 사용하는 Anthropic Claude v2.1 모델을 사용하겠습니다.

## **2. Setup**

### 2.1 Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))

### 2.2 Package Installation
- Install here ==> [package install](../../00_setup/setup.ipynb)

### 2.3. Bedrock Client 생성


In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## **3. Generate SQL Query**
### 3.1. Simple Example
위에서 설명한 사용 사례에 따라 Amazon Bedrock 서비스에서 SQL 쿼리를 생성하기 위한 입력을 준비해 보겠습니다.

In [5]:
# create the prompt to generate SQL query
prompt_data = """

Human: AnyCompany has a database with a table named sales_data containing sales records. The table has following columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Can you generate SQL queries for the below: 
- Identify the top 5 best selling products by total sales for the year 2023
- Calculate the monthly average sales for the year 2023

Assistant:
"""

먼저 Anthorpic Claude v2.1 모델부터 사용해보겠습니다.

앞서 생성한 프롬프트를 기반으로 출력을 생성하는 방법을 살펴봅니다.

In [6]:
modelId = bedrock_info.get_model_id(model_name="Claude-V2-1") # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

In [7]:
# Claude - Body Syntex
body = json.dumps(
    {
        "prompt": prompt_data,
        "max_tokens_to_sample":4096,
        "temperature":0.5,
        "top_k":250,
        "top_p":0.5,
        "stop_sequences": ["\n\nHuman:"]
    }
) 

response = boto3_bedrock.invoke_model(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 Here are the SQL queries for the requested tasks:

```sql
-- Top 5 best selling products by total sales for 2023
SELECT product_id, SUM(price * units_sold) AS total_sales
FROM sales_data
WHERE DATE_FORMAT(date, '%Y') = '2023'
GROUP BY product_id
ORDER BY total_sales DESC
LIMIT 5;

-- Monthly average sales for 2023
SELECT
    DATE_FORMAT(date, '%Y-%m') AS month,
    ROUND(AVG(price * units_sold), 2) AS average_sales
FROM sales_data
WHERE DATE_FORMAT(date, '%Y') = '2023'
GROUP BY month;
```

To explain:

- For the top selling products, we aggregate by `product_id` and sum the total sales, which is
`price * units_sold`. We filter to only 2023 data, group by product, order by total sales descending
and limit to the top 5 results.

- For monthly averages, we extract the month from the `date` field, then average the total sales per
month. We round the average to 2 decimal places and filter to only 2023 data.

Let me know if you need any clarification or have additional requirements!


한글 Query

In [12]:
# create the prompt to generate SQL query
prompt_data_kr = """

Human: AnyCompany has a database with a table named sales_data containing sales records. The table has following columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Can you generate SQL queries for the below: 
- 2023년 총 매출 기준 상위 5개 베스트셀러 제품 파악하기
- 2023년 월평균 매출 계산하기

Assistant:
"""

In [13]:

# Claude - Body Syntex
body = json.dumps(
    {
        "prompt": prompt_data_kr,
        "max_tokens_to_sample":4096,
        "temperature":0.5,
        "top_k":250,
        "top_p":0.5,
        "stop_sequences": ["\n\nHuman:"]
    }
) 

response = boto3_bedrock.invoke_model(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 Here are the SQL queries for the requested tasks:

```sql
-- 2023년 총 매출 기준 상위 5개 베스트셀러 제품 파악하기
SELECT product_id, SUM(price * units_sold) AS total_sales
FROM sales_data
WHERE DATE >= '2023-01-01' AND DATE < '2024-01-01'
GROUP BY product_id
ORDER BY total_sales DESC
LIMIT 5;

-- 2023년 월평균 매출 계산하기
SELECT DATE_FORMAT(date, '%Y-%m') AS year_month,
       ROUND(AVG(price * units_sold), 2) AS avg_monthly_sales
FROM sales_data
WHERE DATE >= '2023-01-01' AND DATE < '2024-01-01'
GROUP BY year_month;
```

The first query aggregates total sales per product for the year 2023, orders them by descending
sales, and limits to the top 5 products to find the top selling products.

The second query calculates the average monthly sales for 2023 by extracting the year and month
parts from the date, grouping by that, and averaging the total sales per month.

Let me know if you need any clarification or have additional requirements to address!


### 3.2. Advanced Example
SQL을 통한 병원의 환자 관리 시스템 이해하기

In [14]:
# create the prompt
prompt_sql_data = """

Human: You're provided with a database schema representing any hospital's patient management system.
The system holds records about patients, their prescriptions, doctors, and the medications prescribed.

Here's the schema:

```sql
CREATE TABLE Patients (
    PatientID int,
    FirstName varchar(50),
    LastName varchar(50),
    DateOfBirth datetime,
    Gender varchar(10),
    PRIMARY KEY (PatientID)
);

CREATE TABLE Doctors (
    DoctorID int,
    FirstName varchar(50),
    LastName varchar(50),
    Specialization varchar(50),
    PRIMARY KEY (DoctorID)
);

CREATE TABLE Prescriptions (
    PrescriptionID int,
    PatientID int,
    DoctorID int,
    DateIssued datetime,
    PRIMARY KEY (PrescriptionID)
);

CREATE TABLE Medications (
    MedicationID int,
    MedicationName varchar(50),
    Dosage varchar(50),
    PRIMARY KEY (MedicationID)
);

CREATE TABLE PrescriptionDetails (
    PrescriptionDetailID int,
    PrescriptionID int,
    MedicationID int,
    Quantity int,
    PRIMARY KEY (PrescriptionDetailID)
);
```

Write a SQL query that fetches all the patients who were prescribed more than 5 different medications on 2023-04-01.

Assistant:
"""

In [15]:
# Claude - Body Syntex
body = json.dumps(
    {
        "prompt": prompt_sql_data,
        "max_tokens_to_sample":4096,
        "temperature":0.5,
        "top_k":250,
        "top_p":0.5,
        "stop_sequences": ["\n\nHuman:"]
    }
) 

response = boto3_bedrock.invoke_model(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 Here is the SQL query to fetch patients who were prescribed more than 5 different medications on
2023-04-01:

```sql
SELECT p.FirstName, p.LastName, COUNT(DISTINCT md.MedicationID) AS num_medications
FROM Patients p
JOIN Prescriptions pre ON p.PatientID = pre.PatientID
JOIN PrescriptionDetails pd ON pre.PrescriptionID = pd.PrescriptionID
JOIN Medications md ON pd.MedicationID = md.MedicationID
WHERE pre.DateIssued = '2023-04-01'
GROUP BY p.FirstName, p.LastName
HAVING COUNT(DISTINCT md.MedicationID) > 5;
```

The key steps are:

1. Join the Patients, Prescriptions, PrescriptionDetails and Medications tables to link all the data
2. Filter to only prescriptions issued on 2023-04-01
3. Group by patient name and count distinct medications prescribed
4. Filter to only patients with more than 5 distinct medication counts

This will give you the patient name and number of distinct medications prescribed to them on the
given date, for those with more than 5 medications.


## **4. Conclusion**
- 아마존 베드락 API 및 boto3 SDK를 사용하여 실험해 보았습니다. 이 API를 사용하여 판매 데이터를 분석하기 위한 SQL 쿼리를 생성하는 사용 사례를 살펴보았습니다.
- **`다양한 프롬프트 엔지니어링 원칙을 적용하여 더 나은 결과를 얻으세요.`** <BR>
    - [Prompting strategies by LangChain](https://python.langchain.com/docs/use_cases/sql/prompting)